In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import json
import os
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
from src.func import tweet_utils
from src.func import regex
from src.func import labmtgen
from src.sentiment.senti_utils import *
from labMTsimple.storyLab import *
import random
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

Read excel file with word frequencies, manually labeled "remove" column
list all remove words by city which are on separate sheets

In [2]:
xls = pd.ExcelFile('./results/city_words.xlsx')
sheets =  pd.read_excel(xls, None)

words_investigate = {}
for city, df in sheets.items():
    words_investigate[city] = list(df[df.remove ==1.0].word.values)

#words_investigate
#set(sum(words_investigate.values(), []))

In [4]:
words_investigate

{'CA_San_Francisco_0667000': ['festival', 'music', 'young', 'flowers'],
 'AZ_Phoenix_0455000': ['garden', 'hospital', 'zoo'],
 'FL_Jacksonville_1235000': ['science', 'museum'],
 'TX_Austin_4805000': ['music', 'limits', 'accident'],
 'CA_San_Diego_0666000': ['zoo', 'sea'],
 'DC_Washington_1150000': ['war', 'bill', 'united', 'health'],
 'WA_Seattle_5363000': ['health', 'surgery', 'emergency'],
 'IL_Chicago_1714000': ['museum', 'music', 'art', 'festival', 'riot'],
 'TX_Houston_4835000': ['hospital',
  'traffic',
  'delay',
  'zoo',
  'accident',
  'science',
  'museum'],
 'OH_Cleveland_3916000': ['garden', 'beach', 'island'],
 'MA_Boston_2507000': ['closed', 'ma', 'partners', 'festival'],
 'NY_New_York_3651000': ['festival', 'natural'],
 'TX_San_Antonio_4865000': ['cafe', 'zoo'],
 'TX_Dallas_4819000': ['festival', 'music', 'health'],
 'PA_Philadelphia_4260000': ['independence', 'art'],
 'CA_Los_Angeles_0644000': ['zoo', 'science', 'festival'],
 'CA_San_Jose_0668000': ['christmas', 'raging

In [5]:
all_stops = ['museum','art','festival','garden','gardens','zoo','closed','traffic','accident','hiring']

In [60]:
### Look at tweets with stop words for given city ###
city = Path("../data/processed/tweets/CA_San_Francisco_0667000.json")
stops = words_investigate[city.stem]
print(stops)
tweets = load_tweets(city)
park_tweets, control_tweets, park_names = park_control_split(tweets)
user_tweets = user_control_split(tweets)

count = 0 
for user, tweets in user_tweets.items():
    if count > 100:
        break
    for tweet in tweets:
        try:
            if  word in tweet['pure_text'].lower():
                #print(word,tweet['pure_text'])
                count+=1
                break
        except TypeError:
            pass

['festival', 'music', 'young', 'flowers']


In [121]:
#### Get hashtags for random set of cities ###

city_list = []
for city in tweet_dir.iterdir():
    city_list.append(city)

hash_cities = random.sample(city_list,5)

park_hts = Counter()
control_hts = Counter()

for city in hash_cities:
    city_words = []
    print("City Hashtags {}".format(city.name))
    tweets = load_tweets(city)
    park_tweets, control_tweets, park_names = park_control_split(tweets)
    park_text = ' '.join([str(tweet['pure_text']).lower() for tweet in park_tweets])
    park_ngrams = dict(regex.get_ngrams(park_text,path='../src/func/ngrams.bin'))
    control_text = ' '.join([str(tweet['pure_text']).lower() for tweet in control_tweets])
    control_ngrams = dict(regex.get_ngrams(control_text,path='../src/func/ngrams.bin'))
    park_hashtags = {}

    for k,v in park_ngrams.items():
        if k[0]=="#" and v > 50:
            park_hashtags[k] = v

    park_hts += Counter(park_hashtags)

    control_hashtags = {}
    for k,v in control_ngrams.items():
        if k[0]=="#" and v > 50:
            control_hashtags[k] = v

    control_hts += Counter(control_hashtags)

[PosixPath('../data/processed/tweets/MI_Detroit_2622000.json'),
 PosixPath('../data/processed/tweets/IN_Indianapolis_1836003.json'),
 PosixPath('../data/processed/tweets/AZ_Phoenix_0455000.json'),
 PosixPath('../data/processed/tweets/DC_Washington_1150000.json'),
 PosixPath('../data/processed/tweets/TX_Houston_4835000.json')]

In [125]:
control_hts.most_common(30)

[('#job', 6223),
 ('#trndnl', 4054),
 ('#hiring', 4027),
 ('#jobs', 3471),
 ('#houston', 2229),
 ('#careerarc', 1758),
 ('#indianapolis', 1609),
 ('#washington', 1480),
 ('#tweetmyjobs', 1047),
 ('#hospitality', 840),
 ('#dc', 818),
 ('#traffic', 724),
 ('#phoenix', 718),
 ('#nursing', 709),
 ('#veterans', 616),
 ('#retail', 545),
 ('#healthcare', 534),
 ('#top3apps', 387),
 ('#washingtondc', 378),
 ('#sales', 326),
 ('#', 292),
 ('#it', 209),
 ('#clerical', 198),
 ('#internship', 182),
 ('#detroit', 181),
 ('#nats', 171),
 ('#businessmgmt', 154),
 ('#photo', 148),
 ('#customerservice', 129),
 ('#accounting', 126)]

In [1]:
### hashtags to remove
hashtags_rm = ["#jobs","#job", "#getalljobs","#hiring", "#tweetmyjobs", "#careerarc","#hospitality", "#healthcare", "#nursing","#marketing", "#sales","#clerical","#it"]

In [129]:
with open("../src/sentiment/hashtags.json", 'w') as fp:
    json.dump(hashtags_rm, fp)